In [48]:
import pandas as pd
import plotly.express as px
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt


In [49]:
print("test")

test


# Analyse van het aantal in zijn geheel

In [50]:
df = pd.read_excel('Totaal_gebruik.xlsx')
df.head()

,Year,Snorfiets,Bromfiets,Totaal
0,2019,716917,387331,1104248
1,2020,728644,388345,1116989
2,2021,763342,393991,1157333
3,2022,781353,404282,1185635
4,2023,761219,436996,1198215


In [51]:
df = df.rename(columns={
    "Year": "Year",
    "Snorfiets": "Snorfiets",
    "Bromfiets": "Bromfiets"
})[["Year", "Snorfiets", "Bromfiets"]]

long = df.melt(id_vars="Year",
                value_vars=["Snorfiets", "Bromfiets"],
                var_name="Serie", value_name="Aantal")


fig = px.line(
    long, x="Year", y="Aantal", color="Serie",
    markers=True,
    labels={"Year": "Jaar", "Aantal": "Aantal", "Serie": ""},
)
fig.update_traces(hovertemplate="Jaar %{x}<br> Aantal %{y:.0f}")
fig.update_layout(title="Aantal gebruikers — Bromfiets vs. Snorfiets", xaxis=dict(dtick=1))
fig.show()



# Analyse van het aantal per leeftijdsgroep

In [5]:
df0 = pd.read_csv('leeftijd_schoon.csv')

In [6]:
df_jaar_leeftijd = (df0.groupby(["Jaar", "Leeftijdscategorie", "Type"])["Aantal"].sum().reset_index())


fig = px.line(
    data_frame = df_jaar_leeftijd,
    x="Jaar",
    y="Aantal",
    color="Leeftijdscategorie",
    facet_col="Type",                 
    markers=True,
    category_orders={"Type": ["Snorfiets", "Bromfiets"]},
    labels={"Jaar": "Jaar", "Aantal": "Aantal", "Leeftijdscategorie": "Leeftijdscategorie"},
    title="Trend per leeftijdscategorie — Snorfietsen vs. Bromfietsen"
)


fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

# Y-as delen voor eerlijke vergelijking
fig.update_yaxes(matches="y")

fig.update_layout(height=500, width=1000, legend_title_text="Leeftijdscategorie")
fig.show()

# Analyse van het aantal per gemeente

In [53]:
df1 = pd.read_excel('Gemeentes.xlsx')

In [54]:
df_jaar_region = (df1.groupby(["Year", "Region", "Type"])["Number"].sum().reset_index())


fig = px.line(
    df_jaar_region, x="Year", y="Number", color="Region",
    facet_col="Type",  
    markers=True
)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

# Y-as delen voor eerlijke vergelijking
fig.update_yaxes(matches="y")

fig.update_layout(height=500, width=1000, legend_title_text="Region")
fig.show()

# Analyse van het aantal doden

In [13]:
df2 = pd.read_excel('Doden.xlsx')

In [14]:
df2 = df2.rename(columns={
    "Perioden": "Year",
    "Totaal": "Total",
    "Brom/Snorfiets": "MopedDeaths"
})[["Year", "Total", "MopedDeaths"]]

long = df2.melt(id_vars="Year",
                value_vars=["Total", "MopedDeaths"],
                var_name="Serie", value_name="Aantal")


fig = px.line(
    long, x="Year", y="Aantal", color="Serie",
    markers=True,
    labels={"Year": "Jaar", "Aantal": "Aantal doden", "Serie": ""},
    category_orders={"Serie": ["Total", "MopedDeaths"]},
)
fig.update_traces(hovertemplate="Jaar %{x}<br>%{y:.0f} doden")
fig.update_layout(title="Verkeersdoden — Totaal vs. Brom/snorfiets", xaxis=dict(dtick=1))
fig.show()

In [15]:

df2 = df2.rename(columns={
    "Perioden": "Year",
    "Totaal": "Total",
    "Brom/Snorfiets": "MopedDeaths"
})[["Year", "Total", "MopedDeaths"]]

long = df2.melt(id_vars="Year",
                value_vars=["Total", "MopedDeaths"],
                var_name="Serie", value_name="Aantal")



# 2) Zorg dat Year/Aantal numeriek zijn
long["Year"] = pd.to_numeric(long["Year"], errors="coerce")
long["Aantal"] = pd.to_numeric(long["Aantal"], errors="coerce")

# 3) Periode-labels + gecombineerde groep
long["Periode"] = np.where(long["Year"] < 2023, "Voor helmplicht", "Na helmplicht")
long["Groep"] = long["Serie"] + " — " + long["Periode"]

# 4) Plot met OLS-trendlijnen per groep
fig = px.scatter(
    long,
    x="Year", y="Aantal",
    color="Groep",               # 4 groepen => 4 trendlijnen
    trendline="ols",
    trendline_scope="trace",     # per kleur-trace
    labels={"Year": "Jaar", "Aantal": "Aantal gebruikers"},
    title="Aantal doden — Bromfiets vs. Snorfiets (voor en na helmplicht)"
)
fig.add_vline(x=2023, line_dash="dash", line_color="red",
              annotation_text="Helmet law inplemented ", annotation_position="top right")


fig.update_layout(xaxis=dict(dtick=1))
fig.show()